In [96]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [97]:
from bokeh.models.widgets import TextInput
from bokeh.layouts import column


def _normaliza(x):
    "traslada a [0,1] los valores de x"
    x = np.array(x)
    minx, maxx = np.min(x), np.max(x)
    return (x-minx) / (maxx-minx)

def jcaos(A, y, marcador):
    "A=vértices a elegir, y=arreglo de índices para esos vértices"
    B = A[y,:] # selecciona los vértices
    # crea una ufunc que calcula el promedio de dos valores (en este caso, puntos 2D)
    prom = np.frompyfunc(lambda x,y:(x+y)/2, 2, 1)
    # Calcula el prom acumulado
    C = prom.accumulate(B, dtype=np.object).astype(np.float)
    x,y = C.T # separa en arreglos ambas coord.
    p = figure(plot_width=600, plot_height=600)
    p.cross(x,y, size=.1, alpha=1)
    show(p)
    
# logística
def itera(iteraciones, r, xinicial, cuantos_valores):
    evalua = np.frompyfunc(lambda x, y: r*x-r*x*x, 2, 1)
    C = evalua.accumulate(np.full((iteraciones,), xinicial), dtype=np.object).astype(np.float)
    return C[-cuantos_valores:]
x0 = itera(100000, 4, 0.3, 1)[0]
xlog = itera(1000000, 4, x0, 1000000)

xu = np.random.rand(1000000)  # aleatorios distr uniforme
xn = np.random.randn(1000000) # aleatorios distr normal
x_lig = pd.read_csv("pach/Serie_ligeros.txt", header=None)[0]
x_pes = pd.read_csv("pach/Serie_pesados.txt", header=None)[0]

def rot(ary, s, c):
    "rota ary theta grados antihorario."
    R = np.matrix([[c, -s], [s, c]])
    return np.array(np.matmul(R,ary))

def eneagono(x, n=5, marcador=",", normaliza=True):
    """Juego del Caos para serie de tiempo x sobre un n-ágono.

    Parámetros:
        x -- serie de tiempo
        n -- número de vértices (default 5)
        marcador -- in [,.ox^-] (default ',')
        normaliza -- False ssi x[i] in [0,1] (default True)
    """
    if normaliza:
        x=_normaliza(x)
    # crea arreglo de 0s, 1s, 2s, 3s como índices para seleccionar vértices
    y = (x*n).astype(np.int8)
    y[y==n]=n-1
    # rotaciones
    A = np.array([[-np.sin(np.pi/n),-np.cos(np.pi/n)]])
    rad=np.radians(360/n)
    s, c = np.sin(rad), np.cos(rad)
    for _ in range(n-1): # los demás vértices
        A = np.append(A, rot(A[-1], s, c), axis=0)
    jcaos(A, y, marcador)

In [84]:
eneagono(xu[:120000], 3)

In [85]:
x = np.arange(0,2*np.pi, .01)
y = np.sin(x)
p = figure(plot_width=600, plot_height=600)
p.cross(x,y, size=.1, alpha=1)
show(p)

In [98]:
def a(grados, desplazamiento, escala):
    g = np.radians(grados)
    mat = np.matrix((np.cos(g), -np.sin(g)), (np.sin(g),np.cos(g)))
    return np.array([mat, desplazamiento, escala])

def menger():
    escala=[1/3,1/3]
    return np.array([a(0,[0,0],escala),
                     a(0,[1,0],escala),
                     a(0,[2,0],escala),
                     a(0,[0,1],escala),
                     a(0,[2,1],escala),
                     a(0,[0,2],escala),
                     a(0,[1,2],escala),
                     a(0,[2,2],escala)])
def grafica(x, ifs):
    print(x)
    n=len(x)
    a = np.array([[0,0]]).repeat(n, axis=0)

    def ea(i, _):
        xi = np.trunc(x[i]*n).astype(np.int8)
        c, d, e = ifs[xi]
        a[i+1,0] = e[0]*(c[0,0]*xi+c[0,1]*y)+d[0]
        a[i+1,1] = e[0]*(c[1,0]*xi+c[1,1]*y)+d[1]
    evalua = np.frompyfunc(ea, 2, 1)
    evalua.accumulate(np.arange(0, len(x), 1, dtype=np.int16), dtype=np.object).astype(float)
    print(a)

In [95]:
ifs = menger()
grafica(np.random.rand(10), menger())

[ 0.98218576  0.03489122  0.68613322  0.74146677  0.84976837  0.85192613
  0.31538001  0.09210493  0.85318427  0.09273284]


IndexError: index 9 is out of bounds for axis 0 with size 8